In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    # path = '~/Documents/GitHub/f1-analytics/'
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [2]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/merged.csv')

In [3]:
### Season to test results

N = 2021

In [4]:
comparison_dict = {
    'model':[],
    'gamma': [],
    'C': [],
    'kernel': [],
    'score': []
    }

In [5]:
data.columns

Index(['season', 'round', 'podium', 'driver_standings_pos',
       'constructor_standings_pos', 'starting_grid', 'driver_points_per_race',
       'constructor_points_per_race', 'driver_points_last_3_races',
       'driver_points_before_race',
       ...
       'circuit_id_rodriguez', 'circuit_id_sepang', 'circuit_id_shanghai',
       'circuit_id_silverstone', 'circuit_id_sochi', 'circuit_id_spa',
       'circuit_id_suzuka', 'circuit_id_villeneuve', 'circuit_id_yas_marina',
       'circuit_id_zandvoort'],
      dtype='object', length=114)

In [6]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season < N]

scaler = StandardScaler()
X_train = train.drop(['podium',], axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.podium.values)

In [7]:
def score_classification(model):
    correct_predictions = 0
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(['podium'], axis=1)
            
        y_test = test.podium

        # Scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # Make Predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns=['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df.sort_values('proba_1', ascending=False, inplace=True)
        prediction_df.reset_index(inplace = True, drop=True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        correct_predictions += precision_score(prediction_df.actual, prediction_df.predicted)

    return correct_predictions / df[df.season == N]['round'].nunique()

In [8]:
# Support Vector Machines

params={'gamma': np.logspace(-4, -1, 20),
        'C': np.logspace(-2, 1, 20),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} 

for gamma in params['gamma']:
    for c in params['C']:
        for kernel in params['kernel']:
            model_params = (gamma, c, kernel)
            model = svm.SVC(probability=True, gamma=gamma, C=c, kernel=kernel)
            model.fit(X_train, y_train)
            
            model_score = score_classification(model)
            
            comparison_dict['model'].append('svm_classifier')
            comparison_dict['gamma'].append(gamma)
            comparison_dict['C'].append(c)
            comparison_dict['kernel'].append(kernel)
            comparison_dict['score'].append(model_score)

In [9]:
comparison_df = pd.DataFrame(comparison_dict)

comparison_df.sort_values('score', ascending=False)

,model,gamma,C,kernel,score
1401,svm_classifier,0.048329,0.379269,poly,0.454545
1197,svm_classifier,0.016238,10.000000,poly,0.454545
1409,svm_classifier,0.048329,0.784760,poly,0.454545
1405,svm_classifier,0.048329,0.545559,poly,0.454545
1261,svm_classifier,0.023357,2.335721,poly,0.454545
...,...,...,...,...,...
643,svm_classifier,0.001833,0.010000,sigmoid,0.000000
644,svm_classifier,0.001833,0.014384,linear,0.000000
646,svm_classifier,0.001833,0.014384,rbf,0.000000
647,svm_classifier,0.001833,0.014384,sigmoid,0.000000


In [1]:
chosen_inputs = comparison_df.query('score > 0.4').sort_values('score', ascending=False)

chosen_inputs.to_csv(path+'parameters/svm_classifier.csv', index=False)

NameError: name 'comparison_df' is not defined

In [11]:
# total = max(df[df.season == N]['round'].unique())
# missing = list(set([i for i in range(1, total + 1)]) - set(df[df.season == N]['round'].unique()))

# print('Missing rounds: {}'.format(missing))